In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aksh7706","key":"c080f097961952c118a108df9a92f513"}'}

In [ ]:

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip /content/the-movies-dataset.zip

unzip:  cannot find or open /content/the-movies-dataset.zip, /content/the-movies-dataset.zip.zip or /content/the-movies-dataset.zip.ZIP.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import ast

In [ ]:
style.use('seaborn-poster')
style.use('ggplot')

In [ ]:
data = pd.read_csv("/content/movies_metadata.csv")
data.head()

In [ ]:
filtered_data = data.filter(['original_title','overview', 'id', 'genres'], axis=1)
len(filtered_data)

45466

In [ ]:
genres = filtered_data['genres'].tolist()

In [ ]:
genre_per_movie = []
genre_dict = {}
invalid_genres = ['Aniplex', 'BROSTA TV', 'Carousel Productions', 'GoHands',
                  'Mardock Scramble Production Committee', 'Odyssey Media',
                  'Pulser Productions', 'Rogue State', 'Sentai Filmworks',
                  'Telescene Film Group Productions', 'The Cartel', 'Vision View Entertainment', 
                  'TV Movie', 'Foreign']
for genre in genres:
  genre = ast.literal_eval(genre)
  genre_list = []
  for val in genre:
    if val['name'] not in invalid_genres:
      genre_list.append(val['name'])
      if val['name'] in genre_dict:
        genre_dict[val['name']] += 1
      else:
        genre_dict[val['name']] = 1
  genre_per_movie.append(genre_list)

genre_dict = {key:val for key, val in genre_dict.items() if val != 1}
genre_dict

{'Action': 6596,
 'Adventure': 3496,
 'Animation': 1935,
 'Comedy': 13182,
 'Crime': 4307,
 'Documentary': 3932,
 'Drama': 20265,
 'Family': 2770,
 'Fantasy': 2313,
 'History': 1398,
 'Horror': 4673,
 'Music': 1598,
 'Mystery': 2467,
 'Romance': 6735,
 'Science Fiction': 3049,
 'Thriller': 7624,
 'War': 1323,
 'Western': 1042}

In [ ]:
len(genre_per_movie)


45466

In [ ]:
filtered_data['genre_list'] = genre_per_movie
filtered_data.head()


,original_title,overview,id,genres,genre_list
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[Animation, Comedy, Family]"
1,Jumanji,When siblings Judy and Peter discover an encha...,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[Romance, Comedy]"
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,11862,"[{'id': 35, 'name': 'Comedy'}]",[Comedy]


In [ ]:
filtered_data = filtered_data[filtered_data['genre_list'].map(lambda d: len(d)) > 0]
len(filtered_data)


42995

In [ ]:
from bs4 import BeautifulSoup
import requests
import os

In [ ]:
!pip install tmdbv3api

In [ ]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
tmdb = TMDb()
tmdb.api_key = 'aaff08090867999bb4cca1f1b268d225'

In [ ]:

def get_posters(url, name):
  try:
    baseurl = 'http://image.tmdb.org/t/p/w185'
    link = baseurl + url
    img = requests.get(link).content
    name = name.replace('/', '_')
    file_name = str(name) + ".jpg"
    path = os.path.join('/content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters', file_name)
    with open(path, 'wb') as handler:
      handler.write(img)
    return path
  except Exception as e:
    print('[ERROR]', str(e))
    return "Error"

In [ ]:

poster_paths = []
movie = Movie()

i = 1
for id, name in zip(filtered_data['id'].tolist(), filtered_data['original_title'].tolist()):
  try:
    m = movie.details(int(id))
    print('Downloading:', name, "i", i)
    poster_paths.append(get_posters(m.poster_path, name))
  except Exception as e:
    print('[ERROR]', str(e))
    poster_paths.append("API Error")
  i += 1
  # temp = name.replace('/', '_')
  # file_name = str(temp) + ".jpg"
  # path = os.path.join('/content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters', file_name)
  # if '/' in name:
  #   print(name)
  #   print("Path:", path)

print(len(poster_paths))

Streaming output truncated to the last 5000 lines.
Downloading: मर्डर 2 i 38281
Downloading: Murder 3 i 38282
Downloading: D'Ardennen i 38283
Downloading: Batman Unlimited: Mechs vs. Mutants i 38284
Downloading: Birdemic 2: The Resurrection i 38285
Downloading: The Search i 38286
Downloading: അക്കരെയക്കരെയക്കരെ i 38287
Downloading: യോദ്ധാ i 38288
Downloading: പിന്നെയും i 38289
Downloading: Black Road i 38290
Downloading: Bachelors i 38291
Downloading: 터널 i 38292
Downloading: Da Xiao Jiang Hu i 38293
Downloading: Little Sister i 38294
Downloading: Certain Women i 38295
Downloading: Hymyilevä mies i 38296
Downloading: Aquarius i 38297
Downloading: Chasing the Devil i 38298
Downloading: The Home Teachers i 38299
Downloading: Mouse in Manhattan i 38300
Downloading: Comedy Central Roast of Denis Leary i 38301
Downloading: Comedy Central Roast of Jeff Foxworthy i 38302
Downloading: Comedy Central Roast of Larry the Cable Guy i 38303
Downloading: Comedy Central Roast of David Hasselhoff i 383

In [ ]:
filtered_data['poster_paths'] = poster_paths
filtered_data.head()

,original_title,overview,id,genres,genre_list,poster_paths
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[Animation, Comedy, Family]",/content/drive/My Drive/Movie_genre_prediction...
1,Jumanji,When siblings Judy and Peter discover an encha...,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[Adventure, Fantasy, Family]",/content/drive/My Drive/Movie_genre_prediction...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[Romance, Comedy]",/content/drive/My Drive/Movie_genre_prediction...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[Comedy, Drama, Romance]",/content/drive/My Drive/Movie_genre_prediction...
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,11862,"[{'id': 35, 'name': 'Comedy'}]",[Comedy],/content/drive/My Drive/Movie_genre_prediction...


In [ ]:
print(len(filtered_data))
print(len(filtered_data[(filtered_data['poster_paths'] == "API Error") | (filtered_data['poster_paths'] == "Error")]))

42995
1926


In [ ]:
filtered_data = filtered_data[filtered_data['poster_paths'] != "API Error"]
print(len(filtered_data))

42651


In [ ]:
filtered_data = filtered_data[filtered_data['poster_paths'] != "Error"]
print(len(filtered_data))

41069


In [ ]:
filtered_data.head()


,original_title,overview,id,genres,genre_list,poster_paths
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[Animation, Comedy, Family]",/content/drive/My Drive/Movie_genre_prediction...
1,Jumanji,When siblings Judy and Peter discover an encha...,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[Adventure, Fantasy, Family]",/content/drive/My Drive/Movie_genre_prediction...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[Romance, Comedy]",/content/drive/My Drive/Movie_genre_prediction...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[Comedy, Drama, Romance]",/content/drive/My Drive/Movie_genre_prediction...
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,11862,"[{'id': 35, 'name': 'Comedy'}]",[Comedy],/content/drive/My Drive/Movie_genre_prediction...


In [ ]:
filtered_data.to_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/dataset_mod.csv")

In [ ]:
filtered_data = pd.read_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/dataset_mod.csv")

In [ ]:

filtered_data.head()

,Unnamed: 0,Unnamed: 0.1,original_title,overview,id,genres,genre_list,poster_paths
0,0,0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","['Animation', 'Comedy', 'Family']",/content/drive/My Drive/Movie_genre_prediction...
1,1,1,Jumanji,When siblings Judy and Peter discover an encha...,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","['Adventure', 'Fantasy', 'Family']",/content/drive/My Drive/Movie_genre_prediction...
2,2,2,Grumpier Old Men,A family wedding reignites the ancient feud be...,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","['Romance', 'Comedy']",/content/drive/My Drive/Movie_genre_prediction...
3,3,3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","['Comedy', 'Drama', 'Romance']",/content/drive/My Drive/Movie_genre_prediction...
4,4,4,Father of the Bride Part II,Just when George Banks has recovered from his ...,11862,"[{'id': 35, 'name': 'Comedy'}]",['Comedy'],/content/drive/My Drive/Movie_genre_prediction...


In [ ]:
len(filtered_data)

41069

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(filtered_data, test_size=0.2)
train, val = train_test_split(train, test_size=0.1)
print(type(train), type(test), type(val))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [ ]:
val.to_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/val_data.csv")
test.to_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/test_data.csv")
train.to_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/train_data.csv")

In [ ]:
import cv2

def resize_img(path):
  try:
    img = cv2.imread(path)
    img = cv2.resize(img, (64, 64))
    img = img.astype(np.float32)/255
    return img
  except Exception as e:
    print(str(e))
    return None


In [ ]:
val_imgs = []
i = 0
for index, row in val.iterrows():
  if i%100 == 0:
    print("Processing i:", i, str(row['poster_paths']))
  img = resize_img(str(row['poster_paths']))
  if img is not None:
    val_imgs.append(img)
  else:
    val = val.drop(index)
    print("Dropping row:", index, "Length:", len(val))
  i += 1

Processing i: 0 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Karov La Bayit.jpg
Processing i: 100 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/The Road to El Dorado.jpg
Processing i: 200 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/The Red Badge of Courage.jpg
Processing i: 300 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Hannah Arendt.jpg
Processing i: 400 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Blonde Venus.jpg
Processing i: 500 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Das Schloß.jpg
Processing i: 600 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Starve.jpg
Processing i: 700 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Skiptrace.jpg
Processing i: 800 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/2081.jpg
Processing i: 90

In [ ]:
assert len(val) == len(val_imgs)
val_np_imgs = np.array(val_imgs)
val_np_imgs.shape

(3286, 64, 64, 3)

In [ ]:
val.to_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/val_data.csv")


In [ ]:
import pickle
with open('/content/drive/My Drive/Movie_genre_prediction_dataset/val_np_img_norm','wb') as f: pickle.dump(val_np_imgs, f)


In [ ]:
train_imgs = []
i = 0
for index, row in train.iterrows():
  if i%100 == 0:
    print("Processing i:", i, str(row['poster_paths']))
  img = resize_img(str(row['poster_paths']))
  if img is not None:
    train_imgs.append(img)
  else:
    train = train.drop(index)
    print("Dropping row:", index, "Length:", len(train))
  i += 1

Processing i: 0 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/État de siège.jpg
Processing i: 100 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Mars Needs Moms.jpg
Processing i: 200 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/唐山大兄.jpg
Processing i: 300 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Picture of Light.jpg
Processing i: 400 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Burn Notice: The Fall of Sam Axe.jpg
Processing i: 500 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Gray Lady Down.jpg
Processing i: 600 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Heart of Stone.jpg
Processing i: 700 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Don't Be a Menace to South Central While Drinking Your Juice in the Hood.jpg
Processing i: 800 /content/drive/My Drive/Movi

In [ ]:
assert len(train) == len(train_imgs)

In [ ]:
train_np_imgs = np.array(train_imgs)
train_np_imgs.shape
train.to_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/train_data.csv")
import pickle
with open('/content/drive/My Drive/Movie_genre_prediction_dataset/train_np_imgs_norm','wb') as f: pickle.dump(train_np_imgs, f)

In [ ]:
test_imgs = []
i = 0
for index, row in test.iterrows():
  if i%100 == 0:
    print("Processing i:", i, str(row['poster_paths']))
  img = resize_img(str(row['poster_paths']))
  if img is not None:
    test_imgs.append(img)
  else:
    test = test.drop(index)
    print("Dropping row:", index, "Length:", len(test))
  i += 1

Processing i: 0 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/同桌的妳.jpg
Processing i: 100 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Five Nights in Maine.jpg
Processing i: 200 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/The Barbarians.jpg
Processing i: 300 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Star Trek II: The Wrath of Khan.jpg
Processing i: 400 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Office Christmas Party.jpg
Processing i: 500 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Tyttö nimeltä Varpu.jpg
Processing i: 600 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/The Emperor's Candlesticks.jpg
Processing i: 700 /content/drive/My Drive/Movie_genre_prediction_dataset/downloaded_posters/Dungeons & Dragons.jpg
Processing i: 800 /content/drive/My Drive/Movie_genre_prediction_dataset

In [ ]:
assert len(test) == len(test_imgs)

In [ ]:
test_np_imgs = np.array(test_imgs)
test_np_imgs.shape

(8214, 64, 64, 3)

In [ ]:
test.to_csv("/content/drive/My Drive/Movie_genre_prediction_dataset/test_data.csv")

In [ ]:
import pickle
with open('/content/drive/My Drive/Movie_genre_prediction_dataset/test_np_imgs_norm','wb') as f: pickle.dump(test_np_imgs, f)